In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [9]:
indep.shape

(400, 4)

In [10]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['none','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda 3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in valida

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['none', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [15]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [16]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9175224000805396


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[72  7]
 [ 3 38]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.96      0.91      0.94        79
           1       0.84      0.93      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.92      0.91       120
weighted avg       0.92      0.92      0.92       120



In [19]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9564680456931152

In [20]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009040,0.007730,0.000000,0.000000,gini,none,10,"{'criterion': 'gini', 'max_features': 'none', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.003868,0.001576,0.000000,0.000000,gini,none,100,"{'criterion': 'gini', 'max_features': 'none', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.641705,0.285035,0.427456,0.208242,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.756757,0.800000,0.717949,0.820513,0.952381,0.809520,0.079737,7
3,0.789347,0.045635,0.036513,0.004876,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.820513,0.809524,0.844444,0.909091,0.952381,0.867191,0.054836,3
4,0.076572,0.009646,0.014444,0.002517,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.777778,0.829268,0.790698,0.829268,0.900000,0.825402,0.042585,6
5,0.742876,0.049278,0.031267,0.007041,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.850000,0.780488,0.844444,0.909091,0.952381,0.867281,0.058895,2
6,0.003299,0.002135,0.000000,0.000000,entropy,none,10,"{'criterion': 'entropy', 'max_features': 'none...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001022,0.000895,0.000000,0.000000,entropy,none,100,"{'criterion': 'entropy', 'max_features': 'none...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.085725,0.009586,0.026306,0.023921,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.722222,0.809524,0.700000,0.829268,0.900000,0.792203,0.073065,8
9,0.709549,0.024445,0.029786,0.003945,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.820513,0.837209,0.818182,0.909091,0.952381,0.867475,0.053837,1
